# Funciones de limpieza del dataset de vinos obtenido en bodeboca.com

Fecha de creacion: 2024/04/15

Ultima fecha de actualizacion de este documento: 2024/04/18 - por Soraya Alvarez Codesal

- **Objetivo general del proyecto** : obtener un motor de recomendacion de vinos de españa

- **Objetivo de este documento**: explorar diferentes opciones para conseguir una funcion unica donde metamos un dataset con los datos obtenidos a partir del web scrapping de la pagina `bodeboca.com` en estado `raw`, y que salga otro dataset/dataframe "limpio", donde hayamos hecho todo el preprocesamiento y que los datos esten listos para utilizarlos tanto en:
    - el modelo de `recomendacion de vinos basado en similitud de cosenos` 
    - como en el `modelo topsis` 
    - Finlamente, todo esto estara integrado en la `web-app de streamlit` que hemos llamado `Wine Me App`
    
    
En este documento hemos unificado los datasets que hemos actualizado de vinos scrapeados de bodeboca.com (`df_vinos_raw`), lo dividimos en 3 partes, uno para cada uno, y actualizamos datos faltantes -NAs- especialmente de rating, notas de cata y maridaje, con datos de vinissimuss.com o bien otras paginas webs. Este dataset se ha nombrado como `df_vinos_raw2`. A continuacion, hemos metido este dataset en la funcion que engloba todo el preprocesamiento del dataset `preprocesamiento_bodeboca(dataset)` y hemos obtenido el dataset `df_vinos_modelos` listo para utilizar en los modelos de recomendacion.

In [8]:
# Cargamos librerias
import pandas as pd
import numpy as np
import re
from collections import Counter
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet


import warnings
warnings.filterwarnings('ignore')

## 1. Cargamos los datasets

In [10]:
# Cargamos los datos
df1 = pd.read_csv('df_vinos_Soraya_v3.csv', encoding='latin1')
df2 = pd.read_csv('df_vinos_Ivan_v3.csv', encoding='latin1')
df3 = pd.read_csv('df_vinos_Maria_v3.csv', encoding='latin1')
df3 = df3.rename(columns={'temp_servir': 'temp_servir '})


# juntamos/concatenamos los 3 datasets
df_vinos1 = pd.concat([df1, df2, df3])

# Para restablecer los índices del DataFrame resultante
df_vinos1 = df_vinos1.reset_index(drop=True)
df_vinos1

,Unnamed: 0,titulo,link,precio,rating,bodega,tipo,grado,anada,variedad,...,volumen_botella,produccion,subzona,nom_vinedo,descripcion,edad_vinedo,rendimiento,cosecha,vinificacion,embotellado
0,22.0,sierra cantabria mÃ¡gico 2016,https://bodeboca.com/vino/sierra-cantabria-mag...,594.00,"5,0",bodegas vinedos-sierra-cantabria,tinto,14.5% vol.,2016.0,"65% tempranillo, 20% garnacha, 4.4% calagraÃ±o...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,30.0,viÃ±a el pisÃ³n 2019,https://bodeboca.com/vino/vina-el-pison-2019,375.00,"5,0",bodegas artadi-vinedos-vinos,tinto,14.5% vol.,2019.0,100% tempranillo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,36.0,artadi la poza de ballesteros 2020,https://bodeboca.com/vino/artadi-la-poza-balle...,105.00,"4,7",bodegas artadi-vinedos-vinos,tinto,14.5% vol.,2020.0,100% tempranillo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,43.0,vermut de categorila,https://bodeboca.com/vino/vermut-categorila,7.25,"5,0",bodegas bisila,red vermouth,14.0% vol.,NaN,"airÃ©n, malvasÃ­a",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,52.0,el puntido 2020 magnum,https://bodeboca.com/vino/el-puntido-2020-magnum,88.90,"5,0",bodegas vinedos-paganos,tinto,14.0% vol.,2020.0,100% tempranillo,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4794,4101.0,tobia selecciÃ³n de autor blanco 2021,https://bodeboca.com/vino/tobia-seleccion-auto...,14.20,NaN,bodegas bodegas-tobia,blanco,13.5% vol.,2021.0,"50% chardonnay, 50% tempranillo blanco",...,botella de 75 cl.,NaN,NaN,NaN,NaN,NaN,entre 5.000 y 9.000 kilogramos por hectÃ¡rea s...,"vendimia manual, seleccionada y escalonada de ...",encubado de la uva despalillada y maceraciÃ³n ...,crianza de 7 meses sobre lÃ­as en las mismas b...
4795,3373.0,viÃ±a magna 14 meses 2020 magnum,https://bodeboca.com/vino/vina-magna-14-meses-...,52.50,"4,8",bodegas dominio-basconcillos,tinto,14.5% vol.,2020.0,"90% tinta del paÃ­s, 10% cabernet sauvignon",...,botella de 150 cl.,NaN,NaN,NaN,los viÃ±edos se encuentran en el punto mÃ¡s al...,NaN,un kilogramos de uva por cepa.,vendimia en cajas de 12 kilogramos.,NaN,crianza de 14 meses en barrica nueva de roble ...
4796,3901.0,carmelo rodero pago de valtarreÃ±a 2020,https://bodeboca.com/vino/carmelo-rodero-pago-...,53.90,"4,3",bodegas bodegas-rodero,tinto,15.0% vol.,2020.0,100% tinta del paÃ­s,...,botella de 75 cl.,NaN,NaN,pago de valtarreÃ±a.,viÃ±edos cultivados en vaso y en espaldera.,entre 45 y 55 aÃ±os.,3.500 kilogramos por hectÃ¡rea.,las uvas son vendimiadas a mano en cajas de 12...,elaboraciÃ³n por gravedad en sistema rotatorio...,crianza de 24 meses en barricas de roble franc...
4797,564.0,el gordo del circo 2022,https://bodeboca.com/vino/el-gordo-circo-2022,15.80,"4,3",productores casa-rojo,blanco,13.0% vol.,2022.0,100% verdejo,...,botella de 75 cl.,NaN,NaN,NaN,"viÃ±edos situados en olmedo, valdestillas y se...",mÃ¡s de 30 aÃ±os.,inferiores a 2.500 kilogramos por hectÃ¡rea.,vendimia manual.,maceraciÃ³n de la uva a baja temperatura duran...,crianza de 5 meses sobre lÃ­as finas.


In [41]:
# Guardamos el DataFrame como un archivo CSV --> raw2
df_vinos1.to_csv('df_vinos_raw2.csv', index=False)

## 2. Funciones de preprocesamiento del dataset de vinos

In [47]:
# Función limpieza de texto
def limpiar_texto(texto):
    '''
    Esta funcion recibe un texto o un dataset y limpia los simbolos que representan letras con acentos o simbolos especiales por los correctos.
    
    Input: texto o dataset
    Output: texto o dataset limpio de simbolos raros por letras apropiadas
    '''
    if isinstance(texto, str):  # Verifica si el valor es una cadena de texto
        texto = texto.replace('Ã¡', 'a').replace('Ã©', 'e').replace('Ã-', 'i').replace('Ã³', 'o').replace('Ãº', 'u').replace('Ã±', 'ñ').replace('Ã¨', 'e').replace('Âº', 'º').replace('Â', 'i').replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'e').replace('ú', 'u').replace('ã', 'i')
        texto = texto.lower().strip()
    return texto


# funcion que extrae las palabras de un texto o columna - especialmente para variedad
def extraer_palabras(texto):
    '''
    Esta funcion recibe un texto y extrae solamente las palabras, usa una expresión regular para encontrar palabras en el texto y elimina los porcentajes
    
    Input: texto con simbolos, numeros ... etc
    Output: texto exclusivamente con letras
    '''
    texto_limpio = re.sub(r'[^a-zA-ZñÑ,\s]', '', texto)
    return texto_limpio


# funcion para Llenar los valores nulos en una columna con la moda de ese grupo
def fillna_mode(x):
    '''
    Esta función, se utiliza para rellenar los valores faltantes en una serie de datos con la moda de esa serie. 
    Si hay más de un valor no nulo en el grupo, se calcula la moda y se rellenan los valores faltantes con él. 
    Si solo hay un valor no nulo o ningún valor no nulo en el grupo, se devuelven los valores originales sin cambios.
    
    Input: recibe un grupo de filas/subset de un dataset, con o sin NANs. Objeto pandas Series, que representa una columna de datos
    Output: objeto pandas Series, donde los valores nulos han sido rellenados con la moda de la columna correspondiente.
    '''
    if len(x.dropna()) > 1:  # Verificar si hay más de un valor no nulo en el grupo
        return x.fillna(x.mode().iloc[0])  # Llenar los nulos con el modo
    else:
        return x  # Si solo tenemos un vino para calcular la moda, mantener los nulos



# Función para etiquetar y lematizar los tokens de una frase utilizando WordNet
def lemmatizer_get_wordnet_pos_phrase(frase_tokens):
    '''
    Esta función toma una lista de tokens de una frase como entrada y devuelve los tokens lematizados junto con sus etiquetas del "Part Of Speech" (POS).
    POS Se refiere a la categoría gramatical a la que pertenece una palabra en un contexto específico. Las etiquetas POS indican si una palabra es un sustantivo, 
    verbo, adjetivo, adverbio, etc. Estas etiquetas son esenciales para comprender la estructura y el significado de una oración en PNL.
    
    Input: Recibe una lista de tokens que representan una frase
    Output: devuelve dos listas: una lista con los tokens lematizados y otra lista con las etiquetas de las partes del discurso correspondientes a cada token.
    '''
    new_tokens = []
    pos_tokens = []

    tags = nltk.pos_tag(frase_tokens)
    wordnet_lemmatizer = WordNetLemmatizer() #cargamos el lematizador
    
    # Definir el diccionario de etiquetas para lematizar
    tag_dict = {"J": wordnet.ADJ,  # Adjetivo
            "N": wordnet.NOUN  # Nombre
           }
    
    for word, tag in tags:
        pos = tag_dict.get(tag[0], wordnet.NOUN)
        new_tokens.append(wordnet_lemmatizer.lemmatize(word, pos))
        pos_tokens.append(pos)
    
    return new_tokens, pos_tokens

    
    
    
# Función para normalizar, lematizar y eliminar stopwords de un texto
def normalizar_lemmatizar(texto):
    '''
    Esta función realiza varias operaciones en un texto dado:
    1. Reemplaza caracteres no alfabéticos por espacios y convierte todo el texto a minúsculas
    2. Tokeniza el texto en palabras: Utiliza nltk.word_tokenize(texto) para dividir el texto en palabras individuales.
    3. Lematiza los tokens y obtiene las etiquetas POS: solo nombres y adjetivos
    4. Filtra stopwords: Elimina las stopwords del castellano
    5. Une los tokens en un solo string y lo devuelve como salida de la función.
    
    Input: Recibe un texto en forma bruta/raw
    Output: devuelve el mismo texto normalizado, limpio de stopwords, tokenizado y lematizado listo para vectorizar
    '''
        
    # Cargar las stopwords y crear el lematizador una sola vez
    stopwords = set(nltk.corpus.stopwords.words('spanish'))
    wordnet_lemmatizer = WordNetLemmatizer()
    
    # Reemplazar caracteres no alfabéticos por espacios y convertir a minúsculas
    texto = re.sub("[^a-zA-ZñÑ]", " ", str(texto)).lower()
    # Tokenizar el texto en palabras
    tokens = nltk.word_tokenize(texto)
    # Lematizar los tokens y obtener las etiquetas POS
    tokens, _ = lemmatizer_get_wordnet_pos_phrase(tokens)
    # Filtrar stopwords
    tokens = [word for word in tokens if word not in stopwords]
    # Unir los tokens en un solo string
    return " ".join(tokens)
   

    
# Funcion que engloba todo el preprocesamiento del dataset de vinos
def preprocesamiento_bodeboca(dataset):
    '''
    Esta funcion carga un dataset con datos de vinos escrapeados de bodeboca y saca otro dataset limpio, preparado para meter en los modelos
    
    Input: dataset con datos de vino de bodeboca.com (dataset_raw)
    Output: dataset limpio despues de todo el preprocesamiento (df_vinos_limpio) 
    
    '''
       
    ##### 1. SELECCIONAMOS LAS COLUMNAS DE INTERESANTES
    # Lista de nombres de columnas que deseamos seleccionar
    columnas_deseadas = ['titulo', 'link', 'precio', 'rating', 'bodega', 'tipo',
                         'grado', 'anada', 'variedad', 'origen', 'vista', 'nariz', 'boca',
                         'temp_servir ', 'maridaje', 'clima', 'suelo', 'envejecimiento']
    # Hacemos el subset
    data = dataset.loc[:, columnas_deseadas]
    
    #### 2.LIMPIAMOS TEXTO DE COLUMNAS STRINGS DE SIMBOLOS RAROS - acentos principalmente
    data = data.applymap(limpiar_texto)
    data = data.applymap(limpiar_texto)
    
    #### 3. REVISAMOS COLUMNAS NUMERICAS:
    # Convertimos rating a float -- numerico 
    data["rating"]=data["rating"].str.replace(",",".")
    data["rating"]=data["rating"].astype(float)
    
    # Sacamos el grado de alcohol de la columna grado
    data["grado"]=data["grado"].str.replace("% vol.","")
    data["grado"]=data["grado"].astype(float)
    
    # Sacamos la temperatura de servir - cogemos la primera temperatura que aparece
    data['temp_servir '] = data['temp_servir '].str.extract(r'(\d{1,2})')
    data["temp_servir "]=data["temp_servir "].astype(float)
    
    #### 4. ELIMINAMOS DUPLICADOS
    data.drop_duplicates(inplace = True)
    
    #### 5. REDUCCION DE TIPOS DE VINO - creamos una nueva columna haciendo un reemplazo de tipos de vino para agrupar en menos tipos
    # Definir el diccionario de reemplazo
    diccionario_reemplazo = {"tinto": "tinto",
                             "red vermouth": "vermouth",
                             "blanco": "blanco",
                             "espumoso": "espumoso",
                             "amontillado": "generoso",
                             "tinto_reserva": "tinto",
                             "tinto reserva": "tinto",
                             'blanco fermentado en barrica': "blanco",
                             'white vermouth': "vermouth",
                             'manzanilla': "generoso",
                             'dulce px': "generoso",                        
                             'palo cortado': "generoso", 
                             'palo cortado vors': "generoso", 
                             'fino': "generoso",                         
                             'rosado' : "rosado",
                             'otro(s)': "generoso", 
                             'tinto joven': "tinto",
                             'tinto crianza': "tinto", 
                             'amontillado vors': "generoso",
                             "oloroso": "generoso",
                             'oloroso vors': "generoso",
                             'aromatised wine': "generoso", 
                             'blanco naturalmente dulce': "generoso",
                             'tinto dulce': "tinto",
                             'blanco dulce': "blanco", 
                             'orange wine': "blanco", 
                             'tinto gran reserva': "tinto", 
                             'cava': "espumoso",
                             'sweet moscatel': "generoso", 
                             'oloroso dulce': "generoso", 
                             'dulce px vors' : "generoso", 
                             'frizzante': "espumoso",
                             'rueda dorado': "blanco", 
                             'vermut dorado': "vermouth", 
                             'dulce': "generoso", 
                             'rancio': "generoso"                         
                            }
    # creacion de columna tipo2 con los nuevos tipos de vino (tinto, blanco, rosado, espumoso, generoso, vermouth)
    data['tipo2'] = data['tipo'].replace(diccionario_reemplazo)
    
    
    #### 6. REVISAMOS LA COLUMNA VARIEDAD - solo extraemos la variedad de uva sin los porcentajes
    data['variedad'] = data['variedad'].astype(str)
    data['variedad2'] = data['variedad'].apply(extraer_palabras)
    
    #### 7. REEMPLAZO DE NANs EN COLUMNAS DE TEXTO/STRING
    # Vamos a utilizar la columna `tipo` original para rellenar los NANs para que tenga sentido con el tipo de vino
    # Primero eliminamos las filas donde el tipo de vino es NA, sino nos da problemas
    data = data.dropna(subset=['tipo'])
    
    # Nulos de clima teniendo en cuenta por agrupacion de tipo de vino    
    data['clima'] = data.groupby('tipo')['clima'].transform(fillna_mode)
    
    # Nulos de suelo teniendo en cuenta por agrupacion de tipo de vino`    
    data['suelo'] = data.groupby('tipo')['suelo'].transform(fillna_mode)
    
    # Nulos de suelo teniendo en cuenta por agrupacion de tipo de vino 
    data['maridaje'] = data.groupby('tipo')['maridaje'].transform(fillna_mode)
    
    
    #### 8. Creamos columnas nuevas agrupando cata y caracteristicas fisicas de la viña
    data["cata"] = data["vista"] + " " + data["nariz"] + " " + data["boca"]
    data ["fisico"] = data["clima"] + " " + data["suelo"]
    
    #### 8.1. REEMPLAZO DE NANs EN columnas cata, fisico y descripcion
    data['cata'] = data.groupby('tipo')['cata'].transform(fillna_mode)
    data['fisico'] = data.groupby('tipo')['fisico'].transform(fillna_mode)
    
    # creamos la columna con descripcion de cata y maridaje
    data["descripcion"] = data["cata"] + " " + data["maridaje"] 
    
    #### 9. Transformaciones de NLP - NORMALIZACION
    # trataremos las columnas: vista, nariz, boca, clima, suelo, maridaje
    # El flujo normal de NLP, sigue los siguientes pasos: Datos --> Normalizacion --> Vectorizacion --> aaplicar Modelo
    # Normalizacion: pasar a minusculas, tokenizar, eliminar stopwords, lematizacion ()   
    #Transformaciones de NLP por columna
    data["descripcion2"] = data.descripcion.apply(normalizar_lemmatizar)
    data["fisico2"] = data.fisico.apply(normalizar_lemmatizar)
    
    # eliminar filas de vinos donde la columna `descripcion2` es 'NAn'
    data = data.dropna(subset=['descripcion2'])
    data = data[data['descripcion2'] != 'nan']
    
    return data

In [48]:
vinos = preprocesamiento_bodeboca(df_vinos1)
vinos

,titulo,link,precio,rating,bodega,tipo,grado,anada,variedad,origen,...,clima,suelo,envejecimiento,tipo2,variedad2,cata,fisico,descripcion,descripcion2,fisico2
0,sierra cantabria magico 2016,https://bodeboca.com/vino/sierra-cantabria-mag...,594.00,5.0,bodegas vinedos-sierra-cantabria,tinto,14.5,2016.0,"65% tempranillo, 20% garnacha, 4.4% calagraño,...",rioja,...,continental con influencia mediterranea,arcillo-limoso / arcillo-arenoso,crianza en barricas nuevas de roble frances.,tinto,"tempranillo, garnacha, calagraño, turrunte...",rojo cereza / reflejos violaceos / capa media-...,continental con influencia mediterranea arcill...,rojo cereza / reflejos violaceos / capa media-...,rojo cereza reflejos violaceos capa medium alt...,continental influencia mediterranea arcillo li...
1,viña el pison 2019,https://bodeboca.com/vino/vina-el-pison-2019,375.00,5.0,bodegas artadi-vinedos-vinos,tinto,14.5,2019.0,100% tempranillo,vino de españa,...,atlantico con influencia continental,arcillo-calcareo,crianza de 8 meses en barricas de roble.,tinto,tempranillo,"bien definidos, nitidos y transparentes cromat...",atlantico con influencia continental arcillo-c...,"bien definidos, nitidos y transparentes cromat...",bien definidos nitidos transparentes cromatica...,atlantico influencia continental arcillo calcareo
2,artadi la poza de ballesteros 2020,https://bodeboca.com/vino/artadi-la-poza-balle...,105.00,4.7,bodegas artadi-vinedos-vinos,tinto,14.5,2020.0,100% tempranillo,vino de españa,...,2020 se inicio con una primavera lluviosa y fr...,arcillo-calcareo con presencia de limo. profun...,NaN,tinto,tempranillo,"colo rojo picota profundo, capa alta, con nota...",2020 se inicio con una primavera lluviosa y fr...,"colo rojo picota profundo, capa alta, con nota...",colo rojo picota profundo capa alta notas viol...,inicio primavera lluviosa fresca creando unas ...
3,vermut de categorila,https://bodeboca.com/vino/vermut-categorila,7.25,5.0,bodegas bisila,red vermouth,14.0,NaN,"airen, malvasi­a",españa,...,atlantico,albariza,NaN,vermouth,"airen, malvasia","color caoba. aroma a hierbas secas, balsamico,...",atlantico albariza,"color caoba. aroma a hierbas secas, balsamico,...",color caoba aroma hierbas secas balsamico espe...,atlantico albariza
4,el puntido 2020 magnum,https://bodeboca.com/vino/el-puntido-2020-magnum,88.90,5.0,bodegas vinedos-paganos,tinto,14.0,2020.0,100% tempranillo,rioja,...,2020 inicia el ciclo anual con abundantes lluv...,franco-arcilloso por su textura y arcillo-calc...,crianza de 15 meses en barrica nueva de roble ...,tinto,tempranillo,"color rojo picota, con ribete dorado. limpio y...",2020 inicia el ciclo anual con abundantes lluv...,"color rojo picota, con ribete dorado. limpio y...",color rojo picota ribete dorado limpio brillan...,inicia ciclo anual abundantes lluvias noviembr...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4794,tobia seleccion de autor blanco 2021,https://bodeboca.com/vino/tobia-seleccion-auto...,14.20,NaN,bodegas bodegas-tobia,blanco,13.5,2021.0,"50% chardonnay, 50% tempranillo blanco",rioja,...,atlantico.,albariza.,crianza de 7 meses sobre li­as en las mismas b...,blanco,"chardonnay, tempranillo blanco",color dorado intenso con reflejos verdosos. in...,atlantico. albariza.,color dorado intenso con reflejos verdosos. in...,color dorado intenso reflejos verdosos intensa...,atlantico albariza
4795,viña magna 14 meses 2020 magnum,https://bodeboca.com/vino/vina-magna-14-meses-...,52.50,4.8,bodegas dominio-basconcillos,tinto,14.5,2020.0,"90% tinta del pai­s, 10% cabernet sauvignon",ribera del duero,...,continental.,arcilloso-calcareo y franco-arenoso.,crianza de 14 meses en barrica nueva de roble ...,tinto,"tinta del pais, cabernet sauvignon",limpio y brillante. exhibe un intenso color ro...,continental. arcilloso-calcareo y franco-arenoso.,limpio y brillante. exhibe un intenso color ro...,limpio brillante exhibe intenso color rojo gra...,continental arcilloso calcareo fr

El dataframe de salida contiene las siguientes columnas:

- titulo: La columna `titulo` contiene el nombre del vino extraido de la pagina bodeboca.com
- link: La columna `link` contiene el link de bodeboca.com para ese vino
- precio: La columna `precio` contiene el el precio en euros de la pagina bodeboca.com para ese vino en las fechas de Abril 2024
- rating: La columna `rating` contiene el rating o valoracion del vino bien por los usuarios de bodeboca.com o de multiples usuarios evaluado en Google u otras paginas de vinos
- bodega: la columna `bodega` contiene el nombre de la bodega a la que pertenece el vino
- tipo: la columna `tipo` contiene la categoria del vino a la que pertenece (), estas pueden ser las siguientes: array(['tinto', 'red vermouth', 'blanco', 'espumoso', 'amontillado', 'oloroso', 'tinto reserva', 'blanco fermentado en barrica', 'white vermouth', 'manzanilla', 'dulce px', 'palo cortado', 'palo cortado vors', 'fino', 'rosado', 'otro(s)', 'tinto joven','tinto crianza', 'amontillado vors', 'oloroso vors','aromatised wine', 'blanco naturalmente dulce', 'tinto dulce', 'blanco dulce', 'orange wine', 'tinto gran reserva', 'cava','sweet moscatel', 'oloroso dulce', 'dulce px vors', 'frizzante','rueda dorado', 'vermut dorado', 'vermouth', 'dulce', 'rancio'],dtype=object)
- grado: la columna `grado` contiene la graduacion en alcohol del vino
- anada: la columna `anada` contiene el año del vino o la añada
- variedad: la columna `variedad` contiene la variedad de uva y sus porcentajes para cada vino
- origen: la columna `origen` contiene informacion del lugar del que procede el vino
- vista: la columna `vista` contiene informacion sobre las caracteristicas visuales del vino
- nariz: la columna `nariz` contiene informacion sobre las caracteristicas olfativas del vino
- boca: la columna `boca` contiene informacion sobre las caracteristicas gustativas del vino
- temp_servir : la columna `temp_servir` contiene informacion sobre recomendaciones de temperatura de servicio del vino
- maridaje: la columna `maridaje` contiene informacion sobre recomendaciones de maridaje/acompanamiento del vino
- clima: la columna `clima` contiene informacion sobre el clima del lugar donde se producen las cepas del vino
- suelo: la columna `suelo` contiene informacion sobre el suelo del lugar donde se producen las cepas del vino
- envejecimiento: la columna `envejecimiento` contiene informacion sobre la crianza del vino
- tipo2: la columna `tipo2` es una variacion de la columna `tipo`. En `tipo2` hemos hecho un reagrupamiento de los tipos en solo 6 clases: tinto, vermouth, blanco, espumoso, generoso, rosado
- variedad2:  la columna `variedad2` es una variacion de la columna `variedad`. En `variedad2` hemos extraido solamente el texto de las variedades de uva y eliminado los %
- cata: la columna `cata` contiene la informacion de cata del vino (boca + nariz + boca)
- fisico: la columna `fisico` contiene la union de la informacion de clima y suelo
- descripcion: la columna `descripcion` contiene la union de la informacion de `cata` + `maridaje`
- descripcion: la columna `descripcion2` contiene la union de la informacion de `cata` + `maridaje` donde hemos eliminado stopwords, normalizado, tokenizado y lematizado el string de cata y maridaje para cada vino (NLP)
- fisico2: la columna `fisco2` es una variacion de la columna `fisico`, donde hemos eliminado stopwords, normalizado, tokenizado y lematizado el string de info fisico para cada vino


Como vemos, las principales columnas numericas, menos el rating, y las principales que usaremos de string, como la columna `descripcion2` tienen 0 datos nulos:

In [49]:
vinos.isna().sum().sort_values()

titulo               0
descripcion          0
cata                 0
variedad2            0
tipo2                0
maridaje             0
descripcion2         0
origen               0
fisico2              0
grado                0
tipo                 0
precio               0
link                 0
variedad             0
clima                1
bodega              16
nariz               25
suelo               27
fisico              27
boca                60
temp_servir        153
anada              462
vista              463
envejecimiento     793
rating            1440
dtype: int64

In [51]:
# Guardamos el DataFrame como un archivo CSV --> dataframe listo para meter en los modelos
vinos.to_csv('df_vinos_modelos.csv', index=False, na_rep="NA")


In [38]:
vinos

,titulo,link,precio,rating,bodega,tipo,grado,anada,variedad,origen,...,clima,suelo,envejecimiento,tipo2,variedad2,cata,fisico,descripcion,descripcion2,fisico2
0,sierra cantabria magico 2016,https://bodeboca.com/vino/sierra-cantabria-mag...,594.00,5.0,bodegas vinedos-sierra-cantabria,tinto,14.5,2016.0,"65% tempranillo, 20% garnacha, 4.4% calagraño,...",rioja,...,continental con influencia mediterranea,arcillo-limoso / arcillo-arenoso,crianza en barricas nuevas de roble frances.,tinto,"tempranillo, garnacha, calagraño, turrunte...",rojo cereza / reflejos violaceos / capa media-...,continental con influencia mediterranea arcill...,rojo cereza / reflejos violaceos / capa media-...,rojo cereza reflejos violaceos capa medium alt...,continental influencia mediterranea arcillo li...
1,viña el pison 2019,https://bodeboca.com/vino/vina-el-pison-2019,375.00,5.0,bodegas artadi-vinedos-vinos,tinto,14.5,2019.0,100% tempranillo,vino de españa,...,atlantico con influencia continental,arcillo-calcareo,crianza de 8 meses en barricas de roble.,tinto,tempranillo,"bien definidos, nitidos y transparentes cromat...",atlantico con influencia continental arcillo-c...,"bien definidos, nitidos y transparentes cromat...",bien definidos nitidos transparentes cromatica...,atlantico influencia continental arcillo calcareo
2,artadi la poza de ballesteros 2020,https://bodeboca.com/vino/artadi-la-poza-balle...,105.00,4.7,bodegas artadi-vinedos-vinos,tinto,14.5,2020.0,100% tempranillo,vino de españa,...,2020 se inicio con una primavera lluviosa y fr...,arcillo-calcareo con presencia de limo. profun...,NaN,tinto,tempranillo,"colo rojo picota profundo, capa alta, con nota...",2020 se inicio con una primavera lluviosa y fr...,"colo rojo picota profundo, capa alta, con nota...",colo rojo picota profundo capa alta notas viol...,inicio primavera lluviosa fresca creando unas ...
3,vermut de categorila,https://bodeboca.com/vino/vermut-categorila,7.25,5.0,bodegas bisila,red vermouth,14.0,NaN,"airen, malvasi­a",españa,...,atlantico,albariza,NaN,vermouth,"airen, malvasia","color caoba. aroma a hierbas secas, balsamico,...",atlantico albariza,"color caoba. aroma a hierbas secas, balsamico,...",color caoba aroma hierbas secas balsamico espe...,atlantico albariza
4,el puntido 2020 magnum,https://bodeboca.com/vino/el-puntido-2020-magnum,88.90,5.0,bodegas vinedos-paganos,tinto,14.0,2020.0,100% tempranillo,rioja,...,2020 inicia el ciclo anual con abundantes lluv...,franco-arcilloso por su textura y arcillo-calc...,crianza de 15 meses en barrica nueva de roble ...,tinto,tempranillo,"color rojo picota, con ribete dorado. limpio y...",2020 inicia el ciclo anual con abundantes lluv...,"color rojo picota, con ribete dorado. limpio y...",color rojo picota ribete dorado limpio brillan...,inicia ciclo anual abundantes lluvias noviembr...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4794,tobia seleccion de autor blanco 2021,https://bodeboca.com/vino/tobia-seleccion-auto...,14.20,NaN,bodegas bodegas-tobia,blanco,13.5,2021.0,"50% chardonnay, 50% tempranillo blanco",rioja,...,atlantico.,albariza.,crianza de 7 meses sobre li­as en las mismas b...,blanco,"chardonnay, tempranillo blanco",color dorado intenso con reflejos verdosos. in...,atlantico. albariza.,color dorado intenso con reflejos verdosos. in...,color dorado intenso reflejos verdosos intensa...,atlantico albariza
4795,viña magna 14 meses 2020 magnum,https://bodeboca.com/vino/vina-magna-14-meses-...,52.50,4.8,bodegas dominio-basconcillos,tinto,14.5,2020.0,"90% tinta del pai­s, 10% cabernet sauvignon",ribera del duero,...,continental.,arcilloso-calcareo y franco-arenoso.,crianza de 14 meses en barrica nueva de roble ...,tinto,"tinta del pais, cabernet sauvignon",limpio y brillante. exhibe un intenso color ro...,continental. arcilloso-calcareo y franco-arenoso.,limpio y brillante. exhibe un intenso color ro...,limpio brillante exhibe intenso color rojo gra...,continental arcilloso calcareo fr